In [1]:
!python /home/jovyan/work/setup.py develop

running develop
running egg_info
writing TemporalTransformer.egg-info/PKG-INFO
writing dependency_links to TemporalTransformer.egg-info/dependency_links.txt
writing top-level names to TemporalTransformer.egg-info/top_level.txt
reading manifest file 'TemporalTransformer.egg-info/SOURCES.txt'
writing manifest file 'TemporalTransformer.egg-info/SOURCES.txt'
running build_ext
Creating /opt/conda/lib/python3.8/site-packages/TemporalTransformer.egg-link (link to .)
TemporalTransformer 0.0.1 is already the active version in easy-install.pth

Installed /home/jovyan/work/TemporalTransformer/notebooks
Processing dependencies for TemporalTransformer==0.0.1
Finished processing dependencies for TemporalTransformer==0.0.1


In [2]:
import sys
import os

os.chdir('/home/jovyan/work/')
os.getcwd()

'/home/jovyan/work'

In [4]:
!pip install tensorflow

     |████████████████████████████████| 320.5 MB 4.6 kB/s  eta 0:00:01  |                                | 624 kB 3.5 MB/s eta 0:01:31     |                                | 901 kB 3.5 MB/s eta 0:01:31     |█████▏                          | 51.2 MB 21.4 MB/s eta 0:00:13     |████████▌                       | 84.6 MB 8.0 MB/s eta 0:00:30     |████████████▏                   | 121.8 MB 8.4 MB/s eta 0:00:24     |█████████████▎                  | 132.9 MB 11.4 MB/s eta 0:00:17     |█████████████▋                  | 136.4 MB 9.3 MB/s eta 0:00:20     |███████████████▍                | 153.8 MB 10.3 MB/s eta 0:00:17     |███████████████▋                | 155.9 MB 7.1 MB/s eta 0:00:24     |█████████████████▍              | 174.4 MB 13.1 MB/s eta 0:00:12     |████████████████████▊           | 207.2 MB 12.7 MB/s eta 0:00:09     |████████████████████▉           | 208.2 MB 12.7 MB/s eta 0:00:09     |██████████████████████▌         | 225.4 MB 4.2 MB/s eta 0:00:23     |█████████████████████████     

In [5]:
from TemporalTransformer import Hopper
from TemporalTransformer import Prepper
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import numpy as np

# Hopper

In [6]:
verbose = True
h = Hopper.dbms(verbose=verbose)

DROP TABLE IF EXISTS i_ids;

CREATE TABLE i_ids (
	i_id TEXT NOT NULL,
	PRIMARY KEY (i_id)
);

DROP TABLE IF EXISTS i_windows;

CREATE TABLE i_windows (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_partitions;

CREATE TABLE i_partitions (
	i_id TEXT NOT NULL,
	partition TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_relcal;

CREATE TABLE i_relcal (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL
);



## Load Tables

In [15]:
ls

LISCENSE.txt    README.md             TemporalTransformer.egg-info/
MANIFEST.in.md  setup.py
__pycache__/    TemporalTransformer/


In [17]:
_tc = Hopper.table_config("characteristics_0", 
                  ["age", "sex", "eth", "bin_ldc", "job"], 
                  ["real", "bin", "ldc", "ldc", "hdc"],
                  has_times=False,
                  primary_key=True)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/characteristics_0.csv", delimiter=',')


_tc = Hopper.table_config("samples_0", 
                  ["SBP", "DBP", "type"], 
                  ["real", "real", "ldc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/samples_0.csv", delimiter=',')



_tc = Hopper.table_config("samples_1", 
                  ["tmps", "hrs", "sbps", "rrs", "dbps", "sats", "wgts"],
                  ["real", "real", 'real', "real", "real", "real", "real"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/samples_1.csv", delimiter=',')


_tc = Hopper.table_config("samples_2", 
                  ["dx"],
                  ["hdc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/samples_2.csv", delimiter=',')



_tc = Hopper.table_config("samples_3", 
                  ["outcome"],
                  ["bin"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/samples_3.csv", delimiter=',')


_tc = Hopper.table_config("samples_4", 
                  ["q"],
                  ["bin"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "/home/jovyan/work/TemporalTransformer/notebooks/data/samples_4.csv", delimiter=',')


DROP TABLE IF EXISTS characteristics_0;

CREATE TABLE characteristics_0 (
	i_id TEXT NOT NULL,
	age REAL NOT NULL,
	sex TEXT NOT NULL,
	eth TEXT NOT NULL,
	bin_ldc TEXT NOT NULL,
	job TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

INSERT OR IGNORE INTO i_ids (i_id) VALUES (?);

INSERT  INTO characteristics_0 (i_id, age, sex, eth, bin_ldc, job) VALUES (?, ?, ?, ?, ?, ?);

CREATE INDEX i_index_characteristics_0_i_id ON characteristics_0(i_id);

CREATE INDEX i_index_characteristics_0_age ON characteristics_0(age);

CREATE INDEX i_index_characteristics_0_sex ON characteristics_0(sex);

CREATE INDEX i_index_characteristics_0_eth ON characteristics_0(eth);

CREATE INDEX i_index_characteristics_0_bin_ldc ON characteristics_0(bin_ldc);

CREATE INDEX i_index_characteristics_0_job ON characteristics_0(job);

10000 rows loaded

DROP TABLE IF EXISTS samples_0;

CREATE TABLE samples_0 (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	SBP RE

In [18]:
h.dew_it(fit_normalization_via_sql_qds=False,
         default_first=0,
         default_last=5)

INSERT  INTO i_windows (i_id, i_st, i_et) VALUES (?, ?, ?);

gen_range_table_sql called: af=None, bf=None
INSERT OR IGNORE INTO i_windows 
SELECT * FROM (
	SELECT i_id, 0 AS i_st, 5 AS i_et
	FROM (
	    SELECT i_id, i_st, i_et
		FROM samples_0
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_0
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_1
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_2
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_3
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_4
		
	)
	GROUP BY i_id
);

INSERT  INTO i_partitions (i_id, partition) VALUES (?, ?);

INSERT OR IGNORE INTO i_partitions 
SELECT * FROM (
	SELECT i_id, CASE 
		 WHEN r <= 0.8 THEN "train" 
		 WHEN r <= 0.9 THEN "dev" 
		 WHEN r <= 1.0 THEN "test" 
		 ELSE "test" 
	END
	FROM (SELECT i_id, ABS(RANDOM())/(9223372036854775807.0) AS r FROM i_ids)
);

CREATE VIEW win_samples_0 AS
    SELECT samples_0.i_id,
	MAX(samples_0.i_st, i_windows.i_st) AS i_st,
	MIN(samples_0.i_et, i_windows.i_et) AS i_et,
	

OperationalError: table win_samples_0 already exists

In [19]:
dbms_save_dict = vars(h)
dbms_save_dict

{'database': ':memory:',
 'verbose': True,
 'con': <sqlite3.Connection at 0x7f299f0a56c0>,
 'cur': <sqlite3.Cursor at 0x7f299efabf80>,
 'cur_man': <TemporalTransformer.sqlite_utils.cursor_manager at 0x7f29cb1b3b50>,
 'mn_tc': <TemporalTransformer.Hopper.table_config at 0x7f29f0754eb0>,
 'mn_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7f29cb1b38e0>,
 'wn_tc': <TemporalTransformer.Hopper.table_config at 0x7f29f074cb80>,
 'wn_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7f29f06df910>,
 'pr_tc': <TemporalTransformer.Hopper.table_config at 0x7f29f0740100>,
 'pr_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7f29cae3ffd0>,
 'rc_tc': <TemporalTransformer.Hopper.table_config at 0x7f29cae3fe50>,
 'rc_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7f29cae3fe80>,
 'windows': True,
 'partitions': True,
 'relcal_set': False,
 'fvms': [<TemporalTransformer.Hopper.flow_view_manager at 0x7f299ef6a460>,
  <TemporalTransformer.Hopper.flow_view_manager at 

In [20]:
for fvm in dbms_save_dict["fvms"]:
    print(fvm.tc.name)

characteristics_0
characteristics_0
characteristics_0
samples_0
characteristics_0
samples_0
samples_1
samples_2
samples_3
samples_4


In [21]:
for fvm in dbms_save_dict["fvms"]:
    print(fvm.nrm_config)

AttributeError: 'flow_view_manager' object has no attribute 'nrm_config'

In [22]:
fvm_save_dicts = []
for fvm in dbms_save_dict["fvms"]:
    
    fvm_tc_save_dict = fvm.tc._get_save_dict()
    fvm_save_dict = {"tc": fvm_tc_save_dict,
                     "filter_config": fvm.filter_config,
                     "nrm_config": fvm.nrm_config}
    
    fvm_save_dicts.append(fvm_save_dict)
    
    #break

AttributeError: 'flow_view_manager' object has no attribute 'nrm_config'

In [23]:
fvm_save_dicts

[]

In [24]:
k = Hopper.dbms(verbose=verbose)

for fvm_save_dict in fvm_save_dicts:
    #_tc = Hopper.table_config.load_save_dict(fvm_save_dict["tc"])
    _tc = Hopper.table_config(**fvm_save_dict["tc"])
    print(vars(_tc))
    k._create_fvm(_tc)

DROP TABLE IF EXISTS i_ids;

CREATE TABLE i_ids (
	i_id TEXT NOT NULL,
	PRIMARY KEY (i_id)
);

DROP TABLE IF EXISTS i_windows;

CREATE TABLE i_windows (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_partitions;

CREATE TABLE i_partitions (
	i_id TEXT NOT NULL,
	partition TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_relcal;

CREATE TABLE i_relcal (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL
);



In [25]:
table_name_fp_dict = {"characteristics_0": "data/characteristics_0.csv",
                      "samples_0": "data/samples_0.csv",
                      "samples_1": "data/samples_1.csv",
                      "samples_2": "data/samples_2.csv",
                      "samples_3": "data/samples_3.csv",
                      "samples_4": "data/samples_4.csv",
                     }

for table_name, data_fp in table_name_fp_dict.items():
    fvm = k.fvm_lookup[table_name]
    k._csv_to_fvm(fvm, data_fp, delimiter=',')

KeyError: 'characteristics_0'

In [ ]:
after_first=None
before_last=None
default_first=None
default_last=None
fit_normalization_via_sql_qds=False


if not k.windows:
    k.set_windows(data=[],
                  after_first=after_first,
                  before_last=before_last,
                  default_first=default_first,
                  default_last=default_last)
    
if not k.partitions:
    k.set_partitions(data=[])
    
k.create_windows_views()
k.create_partition_views_prior_to_filter()

#self.fit_filter() -> set filter
for fvm_save_dict in fvm_save_dicts:
    fvm_name = fvm_save_dict["tc"]["name"]
    fvm_filter_config = fvm_save_dict["filter_config"]
    #print(fvm_name, fvm_filter_config)
    
    fvm = k.fvm_lookup[fvm_name]
    fvm.filter_config = fvm_filter_config

k.set_relcal()
k.fil_agg()

k.create_partition_views_prior_to_normalization() 

#self.fit_normalization(via_sql_qds=fit_normalization_via_sql_qds) -> set nrm
for fvm_save_dict in fvm_save_dicts:
    fvm_name = fvm_save_dict["tc"]["name"]
    fvm_nrm_config = fvm_save_dict["nrm_config"]
    #print(fvm_name, fvm_filter_config)
    
    fvm = k.fvm_lookup[fvm_name]
    fvm.nrm_config = fvm_nrm_config
    
k.normalize()
k.create_final_partition_views()

In [ ]:
tfp_h = Prepper.tf_prepper(h)
tfp_h.features

In [ ]:
tfp_k = Prepper.tf_prepper(h)
tfp_k.features

In [ ]:
for _i, _j in zip(tfp_h.features, tfp_k.features):
    if _i != _j:
        print(_i, _j)

In [ ]:
break

## Window, Filter, Transform, Aggregate, Normalize Data

In [ ]:
#h.dew_it(after_first=35)
h.dew_it(fit_normalization_via_sql_qds=False,
         default_first=0,
         default_last=5)

# Prepper 

In [ ]:
tfp = Prepper.tf_prepper(h)
tfp.features

## Fit Prepper to Desired Task

In [ ]:
tfp.fit(offsets=[0, 1, 2, 4, 5], label_fns=["samples_3/avg_outcome", ], partition="train")

## Make Models

In [ ]:
models = {}

In [ ]:
d0 = tf.keras.layers.Dense(units=32, name="encode")
r0 = tf.keras.layers.LSTM(units=32,return_sequences=True, name="RNN_0")
r1 = tf.keras.layers.LSTM(units=16,return_sequences=True, name="RNN_1")

models["all"] = tfp.build_model(middle_layer_list=[d0, r0, r1])
models["all"].compile(loss="binary_crossentropy")
models["all"].summary()

In [ ]:
d0 = tf.keras.layers.Dense(units=32, name="encode")
r0 = tf.keras.layers.LSTM(units=32,return_sequences=True, name="RNN_0")
r1 = tf.keras.layers.LSTM(units=16,return_sequences=True, name="RNN_1")

models["noWS"] = tfp.build_model(middle_layer_list=[d0, r0, r1], ignore_fns=['samples_4/avg_q', 'samples_4/count'])
models["noWS"].compile(loss="binary_crossentropy")
models["noWS"].summary()

## Make TensorFlow Dataset

In [ ]:
ds = tfp.transform_to_ds()

# Train Model

In [ ]:
models["all"].fit(ds["train"], epochs=1, validation_data=ds["dev"])
#final_model.fit(ds["test"], epochs=1)

In [ ]:
models["noWS"].fit(ds["train"], epochs=1, validation_data=ds["dev"])
#final_wo_model.fit(ds["test"], epochs=1)

# Examine Individual Trajectories

In [ ]:
for idx in ["00000", "00001"]:
    e = Prepper.entity(idx, tfp)
    e.plot()

In [ ]:
for idx in ["00000", "00001"]:
    e = Prepper.entity(idx, tfp)
    print(idx)
    for model_name, model in models.items():
        print(model_name)
        e.predict(model)
        e.plot()

# Operating Characteristics

In [ ]:
partitions = ["train", "dev", "test"]
pop_res = {partition: {} for partition in partitions}

for partition in partitions:
    for model_name, model in models.items():
        print(partition, model_name)
        pop_res[partition][model_name] = \
        Prepper.population(ds[partition], tfp, model)

# Train

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["train"][model_name].roc_curves()
    pop_res["train"][model_name].pr_curves()
    pop_res["train"][model_name].calibration_curves()
    pop_res["train"][model_name].calibration_curves(n_bins=2)
    pop_res["train"][model_name].calibration_curves(quantile=True)
    plt.show()

# Dev

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["dev"][model_name].roc_curves()
    pop_res["dev"][model_name].pr_curves()
    pop_res["dev"][model_name].calibration_curves()
    pop_res["dev"][model_name].calibration_curves(quantile=True)
    plt.show()

# Test

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

# Examine Individual Trajectories

In [ ]:
e = Prepper.entity("00001", tfp)
e.print_labels()
e.plot()
e.predict(models["all"])
e.print_labels()
e.plot()

# Additional Feature Development

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].calc_weights(lambda x: 1/x)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].calc_weights(lambda x: math.log(x)/x)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
len(pop_res["test"]["all"].pred["samples_3/avg_outcome"][0])

In [ ]:
len(pop_res["test"]["all"].weights["samples_3/avg_outcome"][0])

In [ ]:
idx="00000"
s_X, s_Y = tfp.get_specific_XY(idx)

In [ ]:
s_Y_h = models["all"].predict(s_X)

for i, lb in enumerate(tfp.label_fns):
    print(s_Y_h[i].shape, s_Y[lb].shape)

In [ ]:
np.set_printoptions(precision=3)
for fn, v in s_X.items():
    print(fn, "\n", np.squeeze(np.array(v)))

In [ ]:
for i in ds["train"].take(1):
    x, y = i
    for k, v in x.items():
        print(k, '\n', v.shape, '\n')
    
    for i, _y in enumerate(y):
        print("y[%s]" %(i), '\n', _y.shape, '\n')

print("Avg y value: ", tf.reduce_mean(y).numpy())